In [84]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice

In [85]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'

In [86]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [87]:
def bigrams(line): 
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks =  re.sub(r"-\n", " ", notags)
    nodigits =  re.sub(r"\d+", " ", nobreaks)
    noroman =  re.sub(r"\b[XVILMC]+\b", "", nodigits)
    lower =  noroman.lower()
    words_iter =   re.findall(r'\w{2,}', lower)
    bigram_tuples = window(words_iter,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [88]:
t= bigrams("la ma kota")
next(t)
next(t)

'ma kota'

In [89]:
chain.from_iterable(["ala ma kota", "kot ma alę"])

In [90]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items'] 
    year_filtered = filter(lambda item: year in item['judgmentDate'], judgements)
    yield from map(lambda item: item['textContent'], year_filtered)

In [ ]:
json_files = os.listdir(json_data_dir)
judgements_files= filter(lambda name: re.match(filesInYearPattern, name), json_files)
texts =  chain.from_iterable(map(lambda file: judgement_texts(file), judgements_files))
all_bigrams = chain.from_iterable(map(lambda line: bigrams(line),texts))
   
counted_words = Counter(all_bigrams)
counted_words.most_common(10)